In [3]:
# Xuất thư viện
import numpy as np
import cv2
import matplotlib.pyplot as plt

# Đường dẫn tuyệt đối
path = "E:\\Machine learning\\Data\\face_data\\"
ids = range(1, 16) # 15 people
states = ['centerlight', 'glasses', 'happy', 'leftlight', 
          'noglasses', 'normal', 'rightlight','sad', 
          'sleepy', 'surprised', 'wink' ]
prefix = 'subject'
surfix = '.png'     #file extension is png

# open one picture to get the image's size
fn = prefix + '01.' + states[0] + surfix
im = cv2.imread(path + fn, 0)
h = im.shape[0] # hight 
w = im.shape[1] # width
D = h * w
N = len(states)*15
print(N, D, h, w)
X = np.zeros((D, N))

# Tạo nhãn cho dữ liệu, giả sử có 15 người và 11 biểu cảm cho mỗi người
y = np.repeat(range(1, 16), len(states))

# collect all data
count = 0

165 77760 243 320


In [4]:
y_labels = []       # Trạng thái 165 ảnh

# there are 15 people
for person_id in range(1, 16):
    for state in states:
        y_labels.append(state)
        # get name of each image file
        fn = path + prefix + str(person_id).zfill(2) + '.' + state + surfix
        
        # open the file and read as grey image
        tmp = cv2.imread(fn, cv2.IMREAD_GRAYSCALE)
        
        # then add image to dataset X
        X[:, count] = tmp.reshape(D)
        count += 1

print(X)
print(y_labels)

[[130. 130. 130. ... 130. 130. 130.]
 [130. 130. 130. ... 130. 130. 130.]
 [130. 130. 130. ... 130. 130. 130.]
 ...
 [ 68.  68.  68. ...  68.  68.  68.]
 [ 68.  68.  68. ...  68.  68.  68.]
 [ 68.  68.  68. ...  68.  68.  68.]]
['centerlight', 'glasses', 'happy', 'leftlight', 'noglasses', 'normal', 'rightlight', 'sad', 'sleepy', 'surprised', 'wink', 'centerlight', 'glasses', 'happy', 'leftlight', 'noglasses', 'normal', 'rightlight', 'sad', 'sleepy', 'surprised', 'wink', 'centerlight', 'glasses', 'happy', 'leftlight', 'noglasses', 'normal', 'rightlight', 'sad', 'sleepy', 'surprised', 'wink', 'centerlight', 'glasses', 'happy', 'leftlight', 'noglasses', 'normal', 'rightlight', 'sad', 'sleepy', 'surprised', 'wink', 'centerlight', 'glasses', 'happy', 'leftlight', 'noglasses', 'normal', 'rightlight', 'sad', 'sleepy', 'surprised', 'wink', 'centerlight', 'glasses', 'happy', 'leftlight', 'noglasses', 'normal', 'rightlight', 'sad', 'sleepy', 'surprised', 'wink', 'centerlight', 'glasses', 'happy'

In [5]:
# Chia tập dữ liệu thành tập train:test = 8:2
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler

scaler = StandardScaler()
x_scaled = scaler.fit_transform(X.T)
# Chia dữ liệu thành tập huấn luyện và kiểm tra với tỷ lệ 0.8:0.2
X_train, X_test, y_train, y_test = train_test_split(x_scaled, y_labels, test_size=0.2, random_state=42)


# Huấn luyện bằng mô hình CNN

In [6]:
import tensorflow as tf
from sklearn.preprocessing import LabelEncoder
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from sklearn.model_selection import train_test_split
from tensorflow.keras.utils import to_categorical
import numpy as np

# Assuming your data is already loaded and preprocessed (X, y)
# Reshape and normalize
X_reshaped = X.reshape(-1, h, w, 1) / 255.0

# Encode labels
le = LabelEncoder()
y_encoded = le.fit_transform(y_labels)
y_categorical = to_categorical(y_encoded)

# Split the data
X_train, X_test, y_train, y_test = train_test_split(X_reshaped, y_categorical, test_size=0.2, random_state=42)

def gen_model(input_shape, num_classes):
    model = tf.keras.models.Sequential([
        tf.keras.layers.Conv2D(16, (3,3), activation='relu', input_shape=input_shape),
        tf.keras.layers.MaxPooling2D(2, 2),
        tf.keras.layers.Dropout(rate=0.15),
        tf.keras.layers.Conv2D(32, (3,3), activation='relu'),
        tf.keras.layers.MaxPooling2D(2,2),
        tf.keras.layers.Dropout(rate=0.1),
        tf.keras.layers.Conv2D(64, (3,3), activation='relu'),
        tf.keras.layers.MaxPooling2D(2,2),
        tf.keras.layers.Dropout(rate=0.10),
        tf.keras.layers.Flatten(),
        tf.keras.layers.Dense(512, activation='relu'),
        tf.keras.layers.Dense(num_classes, activation='softmax')
    ])

    model.compile(loss='categorical_crossentropy', optimizer="adam", metrics=['acc'])
    return model

# Adjust the input shape and number of classes
model = gen_model((h, w, 1), len(np.unique(y_encoded)))

# Train the model
history = model.fit(X_train, y_train, epochs=10, batch_size=32, validation_data=(X_test, y_test))


c:\Users\admind\python\Lib\site-packages\keras\src\layers\convolutional\base_conv.py:99: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(


Epoch 1/10
5/5 ━━━━━━━━━━━━━━━━━━━━ 9s 1s/step - acc: 0.0713 - loss: 6.5354 - val_acc: 0.0303 - val_loss: 2.7890
Epoch 2/10
5/5 ━━━━━━━━━━━━━━━━━━━━ 5s 902ms/step - acc: 0.1133 - loss: 2.9965 - val_acc: 0.0909 - val_loss: 2.4008
Epoch 3/10
5/5 ━━━━━━━━━━━━━━━━━━━━ 5s 894ms/step - acc: 0.1050 - loss: 2.3885 - val_acc: 0.1212 - val_loss: 2.3988
Epoch 4/10
5/5 ━━━━━━━━━━━━━━━━━━━━ 5s 889ms/step - acc: 0.1326 - loss: 2.3956 - val_acc: 0.0909 - val_loss: 2.4106
Epoch 5/10
5/5 ━━━━━━━━━━━━━━━━━━━━ 5s 894ms/step - acc: 0.1376 - loss: 2.3866 - val_acc: 0.0606 - val_loss: 2.4351
Epoch 6/10
5/5 ━━━━━━━━━━━━━━━━━━━━ 4s 879ms/step - acc: 0.1078 - loss: 2.3702 - val_acc: 0.0909 - val_loss: 2.4148
Epoch 7/10
5/5 ━━━━━━━━━━━━━━━━━━━━ 4s 882ms/step - acc: 0.1043 - loss: 2.3753 - val_acc: 0.0909 - val_loss: 2.4553
Epoch 8/10
5/5 ━━━━━━━━━━━━━━━━━━━━ 5s 888ms/step - acc: 0.1190 - loss: 2.3517 - val_acc: 0.0909 - val_loss: 2.4582
Epoch 9/10
5/5 ━━━━━━━━━━━━━━━━━━━━ 4s 878ms/step - acc: 0.1479 - loss: 2.3

a) Tính độ chính xác của mô hình theo các độ đo Accuracy, Precision và Recall trên cả tập train và test.

In [7]:
from sklearn.metrics import accuracy_score, precision_score, recall_score, classification_report
import numpy as np

# Obtain model predictions
y_pred_train_prob = model.predict(X_train)
y_pred_test_prob = model.predict(X_test)

# Convert probabilities to class labels
y_pred_train = np.argmax(y_pred_train_prob, axis=1)
y_pred_test = np.argmax(y_pred_test_prob, axis=1)

# Convert one-hot encoded true labels back to class labels
y_true_train = np.argmax(y_train, axis=1)
y_true_test = np.argmax(y_test, axis=1)

# Calculate metrics
print("Training Set Metrics:")
print("Accuracy:", accuracy_score(y_true_train, y_pred_train))
print("Precision:", precision_score(y_true_train, y_pred_train, average='macro'))
print("Recall:", recall_score(y_true_train, y_pred_train, average='macro'))
print("\nTest Set Metrics:")
print("Accuracy:", accuracy_score(y_true_test, y_pred_test))
print("Precision:", precision_score(y_true_test, y_pred_test, average='macro'))
print("Recall:", recall_score(y_true_test, y_pred_test, average='macro'))

5/5 ━━━━━━━━━━━━━━━━━━━━ 1s 160ms/step
2/2 ━━━━━━━━━━━━━━━━━━━━ 0s 19ms/step 
Training Set Metrics:
Accuracy: 0.25757575757575757
Precision: 0.3480766019039136
Recall: 0.24600399600399597

Test Set Metrics:
Accuracy: 0.09090909090909091
Precision: 0.12599681020733652
Recall: 0.16666666666666666


c:\Users\admind\python\Lib\site-packages\sklearn\metrics\_classification.py:1509: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))
c:\Users\admind\python\Lib\site-packages\sklearn\metrics\_classification.py:1509: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))


b) Giải thích kết quả thu được:
+ Theo tập train thì Accuracy khá thấp, tương ứng với Precision và Recall. Đây là mô hình khá tệ để thực hiện huấn luyện
+ Để thực hiện test thì Accuracy còn thấp hơn nhiều, khoảng 6% dự đoán được kết quả. Đây là mô hình không tốt để thực hiện phân loại

c) So sánh kết quả khi giảm số chiều với mô hình ANN hoặc Logistic để phân loại 

In [8]:
from sklearn.decomposition import PCA

# Giảm chiều dữ liệu
pca = PCA(n_components=100)
X_reduced = pca.fit_transform(X.T)  # Chú ý là X.T vì chúng ta muốn các hàng của X là các mẫu

In [9]:
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import accuracy_score

# Chia dữ liệu thành tập huấn luyện và kiểm tra với tỷ lệ 0.8:0.2
X_train, X_test, y_train, y_test = train_test_split(X_reduced, y_labels, test_size=0.2, random_state=42)

In [10]:
# Phương pháp Multinomial Logistic Regression
log_reg = LogisticRegression(multi_class='multinomial', max_iter=2000, solver="sag")
log_reg.fit(X_train, y_train)
y_pred_log_reg = log_reg.predict(X_test)
accuracy_log_reg = accuracy_score(y_test, y_pred_log_reg)
print(f'Accuracy: {accuracy_log_reg}')

Accuracy: 0.12121212121212122


c:\Users\admind\python\Lib\site-packages\sklearn\linear_model\_sag.py:350: ConvergenceWarning: The max_iter was reached which means the coef_ did not converge
  warnings.warn(


Kết quả có sự khác nhau:
+ Nếu giảm số chiều rồi xử dụng mô hình Logistic Regression thì độ chính xác đạt 12%
+ Trong khi nếu dùng mô hình CNN thì độ chính xác là 6%
=> Mô hình huấn luyến khá tệ

d) Thực hiện các mô hình CNN và mô hình kết hợp PCA-ANN đã đề cập ở trên, đã được huấn luyện với toàn bộ tập dữ liệu, sau đó chạy dự đoán cho khoảng 05 ảnh chân dung bất kỳ có định dạng như bài thực hành phần ANN (kích thước 320x243, khuôn mặt người ở vị trí hơi lệch bên phải với chiều nhìn vào)

In [11]:
# Phương pháp ANN
from sklearn.neural_network import MLPClassifier

# Khởi tạo mô hình ANN với 1 lớp ẩn có 100 nơ-ron
ann_model = MLPClassifier(hidden_layer_sizes=(100,), random_state=42, max_iter=300)

# Huấn luyện mô hình với tập dữ liệu huấn luyện
ann_model.fit(X_train, y_train)

# Dự đoán nhãn của tập dữ liệu kiểm tra
y_pred = ann_model.predict(X_test)

# Tính toán và in ra độ chính xác của mô hình trên tập kiểm tra
accuracy = accuracy_score(y_test, y_pred)
print("Accuracy:", accuracy)

Accuracy: 0.06060606060606061


In [15]:
# Phân loại ảnh
import os
folder_path = 'D:\\Python\\MachineLearning\\MachineLearning\\week08\\data\\face_data\\'  

num_images = 5

new_height = 243
new_width = 320

X_face_random = np.zeros((new_height * new_width, num_images))

for i in range(num_images):
    image_name = f'Test{i+1}.png'
    image_path = os.path.join(folder_path, image_name)
    tmp = cv2.imread(image_path, cv2.IMREAD_GRAYSCALE)
    resized_image = cv2.resize(tmp, (new_width, new_height))
    X_face_random[:, i] = resized_image.reshape(new_height * new_width)

In [17]:
import cv2
import numpy as np
from sklearn.decomposition import PCA

# Đường dẫn đến 5 ảnh chân dung mới
image_paths = ['D:\\Python\\MachineLearning\\MachineLearning\\week08\\data\\face_data\\test1.png', 
               'D:\\Python\\MachineLearning\\MachineLearning\\week08\\data\\face_data\\test1.png', 
               'D:\\Python\\MachineLearning\\MachineLearning\\week08\\data\\face_data\\test1.png', 
               'D:\\Python\\MachineLearning\\MachineLearning\\week08\\data\\face_data\\test1.png', 
               'D:\\Python\\MachineLearning\\MachineLearning\\week08\\data\\face_data\\test1.png', 
               ]
processed_images = []

for path in image_paths:
    # Đọc ảnh và chỉnh sửa kích thước
    img = cv2.imread(path, cv2.IMREAD_GRAYSCALE)
    resized_img = cv2.resize(img, (243, 320))
    
    # Làm phẳng ảnh và thêm vào danh sách
    flattened_img = resized_img.flatten()
    processed_images.append(flattened_img)

# Chuyển danh sách ảnh thành một NumPy array và giảm chiều
X_new = np.array(processed_images)
X_new_reduced = pca.transform(X_new)  # Sử dụng PCA đã được fit trước đó

# Dự đoán nhóm cho các ảnh mới
predictions = log_reg.predict(X_new_reduced)
print("Dự đoán nhóm cho 5 ảnh mới:", predictions)

Dự đoán nhóm cho 5 ảnh mới: ['noglasses' 'leftlight' 'noglasses' 'leftlight' 'leftlight']
